下載相關資料的小魔法

In [16]:
!curl "https://gist.githubusercontent.com/penut85420/5b383ee875f66cfba70c46ad0e2dd21b/raw/fbd21d468add3ad99370e23ed9dcbf10aa7740c9/kana-spell.json" -o "kana-spell.json"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 80  6881   80  5512    0     0   9261      0 --:--:-- --:--:-- --:--:--  9279
100  6881  100  6881    0     0  11521      0 --:--:-- --:--:-- --:--:-- 11545


匯入相關套件

In [17]:
# %pip install gradio -Uq
import json
import random

import gradio as gr

準備資料

In [18]:
def load_json(file_path):
    with open(file_path, "rt", encoding="UTF-8") as f:
        return json.load(f)


data = load_json("kana-spell.json")

hiragana = data["hiragana"]
katakana = data["katakana"]
category = data["category"]
spell = data["spell"]

開始測驗時選擇題目

In [19]:
def start_test(kana, seion, dakuon, handakuon, yoon):
    # seion 等參數會傳入像是 ["a", "ka"] 等列表
    # 使用 * 將這些列表展開成一維列表
    category = [*seion, *dakuon, *handakuon, *yoon]

    # 從平假名或片假名資料中取出對應的假名，全部放在 char_list 裡面
    char_list = list()
    char_list += [ch for k in category for ch in hiragana[k]] if "平假名" in kana else []
    char_list += [ch for k in category for ch in katakana[k]] if "片假名" in kana else []

    # 如果 char_list 是空的，則拋出錯誤
    if not char_list:
        raise gr.Error("請至少選擇一個類別")

    # 隨機打亂 char_list 的順序
    random.shuffle(char_list)

    # 取出第一個假名
    char = char_list.pop(0)

    # 回傳 char 給 txt_test 用來顯示題目
    # 第一個 char_list 回傳給 st_queue 用來紀錄題目狀態
    # 第二個 char_list 回傳給 debug 用來檢查
    # 回傳 gr.Tabs(selected=1) 來切換到測驗分頁
    return char, char_list, char_list, gr.Tabs(selected=1)

檢查作答是否正確

In [20]:
def check_answer(txt_test, txt_input):
    # 將輸入的拼音轉為小寫並去除前後空白
    txt_input = str.lower(txt_input).strip()

    # 如果 txt_input 符合任何一種拼音，則正確
    if txt_input in spell[txt_test]:
        gr.Info("正確！")

    # 如果拼音不正確，提示使用者正確的答案有哪些可能
    else:
        answer = ", ".join(spell[txt_test])
        gr.Info(f"錯誤，正確答案為 {answer}")

    # 回傳 None 來清空 txt_input 的內容
    return None

顯示下一題

In [21]:
def next_char(st_queue):
    # 若 st_queue 是空的，則顯示測驗結束的訊息
    if not st_queue:
        gr.Info("測驗結束！")
        return None, None, None

    # 繼續從 st_queue 中取出下一個假名
    char = list.pop(st_queue, 0)

    # 分別回傳給 txt_test, st_queue, debug 等元件
    return char, st_queue, st_queue

全選與全不選

In [22]:
def select_all():
    return (
        ["平假名", "片假名"],
        category["seion"],
        category["dakuon"],
        category["handakuon"],
        category["youon"],
    )


def select_none():
    return [], [], [], [], []

In [23]:
def reset():
    return 0, 0, None

In [ ]:
font = gr.themes.GoogleFont("Kiwi Maru")
theme = gr.themes.Ocean(font=font)

with gr.Blocks(theme=theme) as app:
    st_queue = gr.State(None)

    # region define layout
    with gr.Tabs(selected=0) as tabs:
        with gr.Tab(label="設定", id=0):
            with gr.Group():
                chk_kana = gr.CheckboxGroup(["平假名", "片假名"], value=["平假名"], label="假名")
                chk_seion = gr.CheckboxGroup(category["seion"], value=["a"], label="清音")
                with gr.Row():
                    chk_dakuon = gr.CheckboxGroup(category["dakuon"], label="濁音")
                    chk_handakuon = gr.CheckboxGroup(category["handakuon"], label="半濁音")
                chk_youon = gr.CheckboxGroup(category["youon"], label="拗音")

            with gr.Row():
                btn_select_all = gr.Button("全選")
                btn_select_none = gr.Button("全不選")
            btn_start = gr.Button("開始測驗")

        with gr.Tab(label="測驗", id=1):
            with gr.Group():
                with gr.Row():
                    txt_test = gr.Textbox(label="題目", interactive=False)
                    txt_status = gr.Textbox(label="狀態", interactive=False)

                txt_input = gr.Textbox(label="作答", submit_btn=True)

                with gr.Row():
                    n_correct = gr.Number(label="答對題數 ✅", value=0, interactive=False)
                    n_total = gr.Number(label="總答題數 🧮", value=0, interactive=False)

        with gr.Tab(label="紀錄", id=2):
            txt_record = gr.TextArea(show_label=False, interactive=False)

            with gr.Row():
                btn_back_to_settings = gr.Button("回到設定 ⚙️")
                btn_again = gr.Button("再次測驗 🔄")

    debug = gr.TextArea(label="Debug", visible=True)
    # endregion

    # region register events
    btn_start.click(
        start_test,
        [chk_kana, chk_seion, chk_dakuon, chk_handakuon, chk_youon],
        [txt_test, st_queue, debug, tabs],
        show_progress="hidden",
    ).then(reset, outputs=[n_correct, n_total, txt_record])

    txt_input.submit(
        check_answer,
        [txt_test, txt_input],
        txt_input,
        show_progress="hidden",
    ).then(
        next_char,
        st_queue,
        [txt_test, st_queue, debug],
        show_progress="hidden",
    )

    btn_select_all.click(
        select_all,
        outputs=[chk_kana, chk_seion, chk_dakuon, chk_handakuon, chk_youon],
        show_progress="hidden",
    )

    btn_select_none.click(
        select_none,
        outputs=[chk_kana, chk_seion, chk_dakuon, chk_handakuon, chk_youon],
        show_progress="hidden",
    )
    # endregion

    app.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


: 